In [1]:
import pandas as pd
file_path = r"C:\Users\USER\Desktop\3학년 1학기\데이터 마이닝 팀플\여자 데이터\2008년 여자 데이터.csv"
X = pd.read_csv(file_path)

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


# Min-Max Scaler 객체 생성
scaler = MinMaxScaler()

# 선택할 컬럼 리스트
columns_to_scale = ['N_INTK', 'N_EN', 'N_WATER', 'N_PROT', 'N_FAT', 'N_CHO','HE_ht', 'HE_wt',
       'HE_wc' ]

# 선택한 컬럼들에 Min-Max Scaling 적용
X[columns_to_scale] = scaler.fit_transform(X[columns_to_scale])


In [3]:
Y = X['fat']
X = X.drop(['fat', 'DW_WBT_pFT','sex'], axis=1)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle=True, stratify=Y, random_state=42)

In [5]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(random_state=42)
# 특성개수 15개
rfe=RFE(estimator=model, n_features_to_select=15, step=1)
X_selected= rfe.fit_transform(X_train,Y_train)

#그리드 서치 매개변수 그리드
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


# 그리드 서치 객체
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_selected, Y_train)


# 최적의 모델 선택
best_model = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 값
X_test_selected = rfe.transform(X_test)
Y_pred = best_model.predict(X_test_selected)

# 정확도 평가
accuracy = accuracy_score(Y_test, Y_pred)

# 선택된 특성
selected_feature_names = X.columns[rfe.support_]

# 결과 출력
print("Best accuracy:", accuracy)
print("Selected features:", selected_feature_names)
selected_feature_count = len(selected_feature_names)
print("Selected feature count:", selected_feature_count)

Best accuracy: 0.7116666666666667
Selected features: Index(['age', 'EC_lgw_2', 'EC_wht_2', 'BD1_11', 'BP8', 'HW_MS_A', 'N_INTK',
       'N_EN', 'N_PROT', 'N_FAT', 'N_CHO', 'HE_ht', 'HE_wc', 'DW_WBT_BMD',
       'BMI'],
      dtype='object')
Selected feature count: 15
